In [ ]:
from data_loader import *
from utils import *

In [ ]:
find_team("Lei")

In [ ]:
teams_data = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")

In [ ]:
unique_teams = teams_data['team_name'].unique()

In [ ]:
teams = unique_teams
season = 2023
visited_list = list()
for team in teams:
    try:
        dat = read_fixtures_for_season(team,season)
        visited_list.append(team)
    except:
        print(f"{team} does not have matches in {season}")

processing for Angers, 2023
Pulling for Angers,2023 with league_id: 61
Angers does not have matches in 2023
processing for Lille, 2023
Pulling for Lille,2023 with league_id: 61


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals_scored'] = fixtures[(fixtures.fixture_id == fixture)]['goals_home'].values[0] - fixtures[(fixtures.fixture_id == fixture)]['score_penalty_home'].fillna(0).values[0]
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:106: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fixture_dat_expanded['team_non_penalty_goals